In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import numpy as np
import requests
import pathlib
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
url = 'https://0245-71-92-86-252.ngrok-free.app/base.keras'
r = requests.get(url)
open('base.keras', 'wb').write(r.content)

In [ ]:
dir_name = input('Enter Data Folder Name: ')
data_dir = pathlib.Path(dir_name)
image_size = (256, 256)
all_images = list(data_dir.glob('*/*.jpg'))

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
categories = list(data_dir.glob('*/'))

In [ ]:
dataset_dict = {}
os.mkdir('resized')
for (category_index, category) in enumerate(categories):
	os.mkdir(pathlib.Path('resized') / category.relative_to(data_dir))
	category_dir = list(category.glob('*.jpg'))
	dataset_dict[category.name] = []
	for (image_index, image) in enumerate(category_dir):
		with Image.open(image) as img:
			print(f"Working on {image.name} in {category.name} (Image {image_index+1}/{len(category_dir)} of Category {category_index+1}/{len(categories)})")
			resized_img = ImageOps.fit(img, image_size)
			resized_img.save(pathlib.Path('resized') / image.relative_to(data_dir))
	print("Finished!")

In [ ]:
data_dir = pathlib.Path(dir_name).parent / 'resized'

In [ ]:
batch_size = 32
img_height = 256
img_width = 256

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
	data_dir,
	validation_split=0.2,
	subset="training",
	seed=123,
	image_size=(img_height, img_width),
	batch_size=batch_size
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
for image_batch, labels_batch in train_ds:
	print(image_batch.shape)
	print(labels_batch.shape)
	break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = len(class_names)

model = keras.saving.load_model("base.keras")

In [ ]:
model.compile(optimizer='adam',
	loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
	metrics=['accuracy']
)

In [ ]:
epochs=10
history = model.fit(
	train_ds,
	validation_data=val_ds,
	epochs=epochs
)

In [ ]:
model.save('addition.keras')